<a href="https://colab.research.google.com/github/ZahraFarahzadii/PV-fault-detection-FewShot-Learning/blob/main/fewshot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !zip -r /content/LibFewShot.zip /content/LibFewShot

In [ ]:
!gdown 16OiH7HGY0JFUUFj5sarpi42C1Xvp2KP4
!unzip /content/LibFewShot.zip

Streaming output truncated to the last 5000 lines.
  inflating: content/LibFewShot/dataset0/images/n0314621900000514.jpg  
  inflating: content/LibFewShot/dataset0/images/n0211454800000531.jpg  
  inflating: content/LibFewShot/dataset0/images/n0377043900000554.jpg  
  inflating: content/LibFewShot/dataset0/images/n0327201000000538.jpg  
  inflating: content/LibFewShot/dataset0/images/n0274717700000115.jpg  
  inflating: content/LibFewShot/dataset0/images/n0425813800000807.jpg  
  inflating: content/LibFewShot/dataset0/images/n0153282900000034.jpg  
  inflating: content/LibFewShot/dataset0/images/n1305456000000513.jpg  
  inflating: content/LibFewShot/dataset0/images/n0279516900000337.jpg  
  inflating: content/LibFewShot/dataset0/images/n0429656200000866.jpg  
  inflating: content/LibFewShot/dataset0/images/n0153282900000811.jpg  
  inflating: content/LibFewShot/dataset0/images/n0414661400001262.jpg  
  inflating: content/LibFewShot/dataset0/images/n0390861800000740.jpg  
  inflating: 

In [ ]:
!mv /content/content/LibFewShot  /content/LibFewShot
!rmdir /content/content

In [ ]:
import os
os.chdir('/content/LibFewShot')
# os.chdir('/content')
os.getcwd()

'/content/LibFewShot'

In [ ]:
!pip install -r requirements.txt
!pip install scikit-learn-intelex

In [ ]:
!pip install torchsummary

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary

In [ ]:
class Conv64F(nn.Module):
  def __init__(
        self,
        is_flatten=False,
        is_feature=False,
        leaky_relu=False,
        negative_slope=0.2,
        last_pool=True,
        maxpool_last2=True,
    ):
        super(Conv64F, self).__init__()

        self.is_flatten = is_flatten
        self.is_feature = is_feature
        self.last_pool = last_pool
        self.maxpool_last2 = maxpool_last2

        if leaky_relu:
            activation = nn.LeakyReLU(negative_slope=negative_slope, inplace=True)
        else:
            activation = nn.ReLU(inplace=True)

        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            activation,
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            activation,
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            activation,
        )
        self.layer3_maxpool = nn.MaxPool2d(kernel_size=2, stride=2)

        self.layer4 = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            activation,
        )
        self.layer4_pool = nn.MaxPool2d(kernel_size=2, stride=2)

  def forward(self, x):
        out1 = self.layer1(x)
        out2 = self.layer2(out1)
        out3 = self.layer3(out2)

        if self.maxpool_last2:
            out3 = self.layer3_maxpool(out3)
            # for some methods (relation net etc.)

        out4 = self.layer4(out3)
        if self.last_pool:
            out4 = self.layer4_pool(out4)

        if self.is_flatten:
            out4 = out4.view(out4.size(0), -1)

        if self.is_feature:
            return out1, out2, out3, out4

        return out4


device= torch.device("cuda" if torch.cuda.is_available()else "cpu")
model=Conv64F().to(device)
summary(model, (3,84,84))


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 84, 84]           1,792
       BatchNorm2d-2           [-1, 64, 84, 84]             128
              ReLU-3           [-1, 64, 84, 84]               0
              ReLU-4           [-1, 64, 84, 84]               0
              ReLU-5           [-1, 64, 84, 84]               0
              ReLU-6           [-1, 64, 84, 84]               0
         MaxPool2d-7           [-1, 64, 42, 42]               0
            Conv2d-8           [-1, 64, 42, 42]          36,928
       BatchNorm2d-9           [-1, 64, 42, 42]             128
             ReLU-10           [-1, 64, 42, 42]               0
             ReLU-11           [-1, 64, 42, 42]               0
             ReLU-12           [-1, 64, 42, 42]               0
             ReLU-13           [-1, 64, 42, 42]               0
        MaxPool2d-14           [-1, 64,